# 2장 파이썬 속성 강좌

## 2.2 기본기에서 한 걸음 나아가기

### 2.2.7 함수형 도구

In [5]:
def exp(base, power):
    return base ** power

밑을 2로 하는 지수 함수를 정의할 때 아래와 같이 <font color='red'>기존 함수를 호출</font>하는 방식으로 정의할 수 있습니다.

In [6]:
def two_to_the_by_call(power):
    return exp(2, power)

또는 functools.partial을 이용하면 기존 함수의 특정 부분을 이용하여 <font color='red'>새로운 함수를 만드는 것</font>이 가능합니다.

In [7]:
from functools import partial

two_to_the_by_new = partial(exp, 2)
print(two_to_the_by_new(3))

8


`exp()` 함수를 수정하고 실행해 보면 호출 방식과 새로운 함수 생성 방식의 차이를 확인할 수 있습니다.

In [8]:
def exp(base, power):
    return base ** power + 1

print(two_to_the_by_call(3))
print(two_to_the_by_new(3))

9
8


# 4장 선형대수

## 4.1 벡터

In [19]:
v = [1, 2, 3]
w = [4, 5, 6]

def vector_add(v, w):
    return [v_i + w_i for v_i, w_i in zip(v, w)]

vector_add(v, w)

[5, 7, 9]

In [20]:
def vector_subtract(v, w):
    return [v_i - w_i for v_i, w_i in zip(v, w)]

vector_subtract(v, w)

[-3, -3, -3]

In [29]:
from functools import reduce

def vector_sum(vectors):
    return reduce(vector_add, vectors)
    
vector_sum([v, w])

[5, 7, 9]

In [31]:
from functools import partial

vector_sum = partial(reduce, vector_add)

vector_sum([v, w])

[5, 7, 9]

In [33]:
def scalar_multiply(c, v):
    return [c * v_i for v_i in v]

In [34]:
def vector_mean(vectors):
    return scalar_multiply(1/len(vectors), vector_sum(vectors))

vector_mean([v, w])

[2.5, 3.5, 4.5]

In [1]:
v1 = [1, 2, 3]
v2 = [4, 5, 6]

def dot(v, w):
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

dot(v1, v2)

32

In [2]:
def sum_of_squares(v):
    return dot(v, v)

import math

def magnitude(v):
    return math.sqrt(dot(v, v))

In [38]:
def distance(v, w):
    return magnitude(vector_subtract(v, w))

## 4.2 행렬

In [3]:
A = [[1, 2, 3],
     [4, 5, 6]]

In [4]:
def shape(A):
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols

In [5]:
def get_row(A, i):
    return A[i]

def get_column(A, j):
    return [A_i[j] for A_i in A]

In [6]:
def make_matrix(num_rows, num_cols, entry_fn):
    [
        [entry_fn(i, j) for j in range(num_cols)] 
        for i in range(num_rows)
    ]

# 5장 통계

## 5.1 데이터셋 설명하기

### 5.1.1 중심 경향성

In [7]:
num_friends = [100, 49, 41, 40, 25]

In [8]:
def mean(x):
    return sum(x) / len(x)

mean(num_friends)

51.0

### 5.1.2 산포도

In [9]:
def de_mean(x):
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

de_mean(num_friends)

[49.0, -2.0, -10.0, -11.0, -26.0]

In [10]:
def variance(x):
    n = len(x)
    deviations = de_mean(x)
    return sum([x_i * x_i for x_i in de_mean(x)]) / len(x)

variance(num_friends)

660.4

In [11]:
import math

def standard_deviation(x):
    return math.sqrt(variance(x))
    
standard_deviation(num_friends)

25.69824896758532

# 8장 경사 하강법

## 8.3 Gradient 적용하기

In [39]:
import random

def step(v, direction, step_size):
    return [v_i + step_size * direction_i for v_i, direction_i in zip(v, direction)]

def sum_of_squares_gradient(v):
    return [2 * v_i for v_i in v]

v = [random.randint(-10, 10) for i in range(3)]

tolerance = 0.0000001

while True:
    gradient = sum_of_squares_gradient(v)
    next_v = step(v, gradient, -0.01)
    if distance(next_v, v) < tolerance:
        break
    v = next_v

## 8.4 적절한 이동 거리 정하기

In [ ]:
def safe(f):
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')
    
    return safe_f

## 8.5 종합하기

In [35]:
def minimize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    
    theta = theta_0
    target_fn = safe(target_fn)
    value = target_fn(theta)
    
    while True:
        gradient = gradient_fn(theta)
        next_thetas = [step(theta, gradient, -step_size) for step_size in step_sizes]
        
        next_theta = min(next_thetas, key=target_fn)
        next_value = target_fn(next_theta)
        
        if abs(value - next_value) < tolerance:
            return theta
        else:
            theta, value = next_theta, next_value

# 10장 데이터 다루기

## 10.1 데이터 탐색하기

## 10.4 척도 조절

In [12]:
# 각 열의 평균과 표준편차를 반환
def scale(data_matrix):
    num_rows, num_cols = shape(data_matrix)
    means = [mean(get_column(data_matrix, j)) for j in range(num_cols)]
    stdevs = [standard_devication(get_column(data_matrix, j)) for j in range(num_cols)]
    return means, stdevs

## 10.5 차원 축소

In [13]:
def de_mean_matrix(A):
    nr, nc = shape(A)
    column_means, _ = scale(A)
    return make_matrix(nr, nc, lambda i, j: A[i][j] - column_means[j])

In [14]:
def direction(w):
    mag = magnitude(w)
    return [w_i / mag for w_i in w]

In [15]:
def directional_variance_i(x_i, w):
    return dot(x_i, w) ** 2

def directional_variance(X, w):
    return sum(directional_variance_i(x_i, w) for x_i in X)